In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::320294666948:role/AWSGlueServiceRole-Lab4
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 5ab0692c-c1d9-4e21-8de0-581d53128fc5
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
Waiting for session 5ab0692c-c1d9-4e21-8de0-581d53128fc5 to get into ready status...
Session 5ab0692c-c1d9-4e21-8de0-581d53128fc5 has been created.




- Na célula a seguir, adicione o código necessário para ler o conteúdo do arquivo _nomes.csv_ para um dataframe Spark.



In [31]:
# Adicione seu código aqui

df = spark.read.csv('s3://sprint7nomes/lab-glue/input/nomes.csv', header=True, inferSchema=True)
df.show()

+---------+----+-----+----+
|     nome|sexo|total| ano|
+---------+----+-----+----+
| Jennifer|   F|54336|1983|
|  Jessica|   F|45278|1983|
|   Amanda|   F|33752|1983|
|   Ashley|   F|33292|1983|
|    Sarah|   F|27228|1983|
|  Melissa|   F|23472|1983|
|   Nicole|   F|22392|1983|
|Stephanie|   F|22323|1983|
|  Heather|   F|20749|1983|
|Elizabeth|   F|19838|1983|
|  Crystal|   F|17904|1983|
|      Amy|   F|17095|1983|
| Michelle|   F|16828|1983|
|  Tiffany|   F|15960|1983|
| Kimberly|   F|15374|1983|
|Christina|   F|15359|1983|
|    Amber|   F|14886|1983|
|     Erin|   F|14835|1983|
|  Rebecca|   F|14711|1983|
|   Rachel|   F|14588|1983|
+---------+----+-----+----+
only showing top 20 rows


- Imprima o esquema do dataframe

In [32]:
# Adicione seu código aqui
df.printSchema()

root
 |-- nome: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- total: integer (nullable = true)
 |-- ano: integer (nullable = true)


- Na próxima célula, escreva o código necessário para alterar a caixa dos valores da coluna _nome_ para MAIÚSCULO


In [33]:
# Adicione seu código aqui
from pyspark.sql.functions import upper
#df = spark.read.csv('s3://sprint7nomes/lab-glue/input/nomes.csv', header=True, inferSchema=True)
df = df.withColumn("nome", upper(df['nome']))
df.show()

+---------+----+-----+----+
|     nome|sexo|total| ano|
+---------+----+-----+----+
| JENNIFER|   F|54336|1983|
|  JESSICA|   F|45278|1983|
|   AMANDA|   F|33752|1983|
|   ASHLEY|   F|33292|1983|
|    SARAH|   F|27228|1983|
|  MELISSA|   F|23472|1983|
|   NICOLE|   F|22392|1983|
|STEPHANIE|   F|22323|1983|
|  HEATHER|   F|20749|1983|
|ELIZABETH|   F|19838|1983|
|  CRYSTAL|   F|17904|1983|
|      AMY|   F|17095|1983|
| MICHELLE|   F|16828|1983|
|  TIFFANY|   F|15960|1983|
| KIMBERLY|   F|15374|1983|
|CHRISTINA|   F|15359|1983|
|    AMBER|   F|14886|1983|
|     ERIN|   F|14835|1983|
|  REBECCA|   F|14711|1983|
|   RACHEL|   F|14588|1983|
+---------+----+-----+----+
only showing top 20 rows


- Apresente a contagem de linhas contidas no dataframe

In [34]:
# Adicione seu código aqui
contagem = df.count()
print(contagem)

1825433


- Agora, apresente a contagem de nomes, agrupando os dados do dataframe pelas colunas _ano_ e _sexo_. Ordene os dados de modo que o ano mais recente aparece como primeiro registro do dataframe.

In [35]:
# Adicione seu código aqui
from pyspark.sql.functions import col
contagem_nomes = df.groupBy("ano", "sexo").agg({"nome": "count"})
contagem_nomes = contagem_nomes.orderBy(col("ano").desc())
name_counts.show()

+----+----+-----------+
| ano|sexo|count(nome)|
+----+----+-----------+
|2014|   M|      13977|
|2014|   F|      19067|
|2013|   M|      14012|
|2013|   F|      19191|
|2012|   M|      14216|
|2012|   F|      19468|
|2011|   M|      14329|
|2011|   F|      19540|
|2010|   M|      14241|
|2010|   F|      19800|
|2009|   F|      20165|
|2009|   M|      14519|
|2008|   M|      14606|
|2008|   F|      20439|
|2007|   M|      14383|
|2007|   F|      20548|
|2006|   M|      14026|
|2006|   F|      20043|
|2005|   M|      13358|
|2005|   F|      19175|
+----+----+-----------+
only showing top 20 rows


- Qual foi o nome feminino com mais registro e em que ano?

In [36]:

# Adicione seu código aqui
from pyspark.sql.functions import desc
df_feminino = df.filter(df.sexo == 'F').groupBy('nome').count().orderBy(desc('count')).first()
nome_feminino, count_feminino = df_feminino[0], df_feminino[1]
ano = df.filter((df.sexo == 'F') & (df.nome == nome_feminino)).groupBy('ano').count().orderBy(desc('count')).first()[0]
print(nome_feminino)
print(ano)

SARAH
1936


- Qual foi o nome masculino com mais registro e em que ano?

In [37]:
# Adicione seu código aqui

df_masculino = df.filter(df.sexo == 'M').groupBy('nome').count().orderBy(desc('count')).first()
nome_masculino, count_masculino = df_masculino[0], df_masculino[1]
ano_masc = df.filter((df.sexo == 'M') & (df.nome == nome_masculino)).groupBy('ano').count().orderBy(desc('count')).first()[0]
print(nome_masculino)
print(ano_masc)


HARRISON
1955


- Apresente o total de registros (masculinos e femininos) para cada ano presente no dataframe. Considere apenas as primeiras 10 linhas, ordenadas pelo ano, de forma crescente.

In [38]:
# Adicione seu código aqui

from pyspark.sql.functions import asc
df_total = df.groupBy('ano', 'sexo').count().orderBy(asc('ano')).limit(10)
df_total.show()

+----+----+-----+
| ano|sexo|count|
+----+----+-----+
|1880|   M| 1058|
|1880|   F|  942|
|1881|   F|  938|
|1881|   M|  997|
|1882|   F| 1028|
|1882|   M| 1099|
|1883|   M| 1030|
|1883|   F| 1054|
|1884|   F| 1172|
|1884|   M| 1125|
+----+----+-----+


- Escreva o conteúdo do dataframe com os valores de nome em maiúsculo no S3. Atenção aos requisitos:
  - A gravação deve ocorrer no subdiretório _frequencia_registro_nomes_eua_ do path s3://\<BUCKET\>/lab-glue/
  - O formato deve ser JSON
  - O particionamento deverá ser realizado pelas colunas _sexo_ e _ano_ (nesta ordem)

In [39]:
# Adicione seu código aqui
df_json = df.withColumn('nome', upper(df['nome']))
df_json.write.partitionBy('sexo', 'ano').json("s3://sprint7nomes/lab-glue/frequencia_registro_nomes_eua")

In [40]:
job.commit()